### <b> Rail Operations Power BI Integration Testing </b>
##### -Building and testing scripts in IDE then importing into PBI power query for transformation of report data <br> -Notebook is categorized by scripts for each report respectively

In [9]:
# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Remove row and column display limits
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Date Test Function
##### Testing to build script that filters data for train operational day, this is because scripts are intended for live refreshed connection into Databricks or cloud service, thus in order to increase computation efficiency, filter for data produced only on the specified day instead of extracting everything

In [10]:
# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Insert data 
dataset = pd.read_excel("../src/rail_sample_static_data.xlsx", sheet_name="Date_Test")

# Convert "date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Create df to view data for the given day
filtered_dataset = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

# Output the result
filtered_dataset

,Date,Role,Duty ID,Trip ID,Starting Location,Start Time,Ending Location,End Time,Start Route Pattern,End Route Pattern,Planning Level,Forename,Surname,Personnel ID,Duty Class


### Number of Trainees Report
##### Testing to build sample of trainee metrics, filtering the dataset for specified operational day, then drilling further for trainee information, then aggregating duty and trip information to calculate trainee measures

In [11]:
# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Duty_Crew_Stats.xlsx", sheet_name="Sheet1")

# Concat for full name field
dataset["Full Name"] = dataset["Surname"] + ", " + dataset["Forename"]

# Change field to str for groupby
dataset["Trip ID"] = dataset["Trip ID"].astype(str)

# Convert "date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Create df to view data for the given day
filtered_dataset = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

filtered_dataset = filtered_dataset[(filtered_dataset["Trip ID"].notna()) & (filtered_dataset["Personnel ID"].notna())]

# Filter for only trainee duties
keywords = ["TRNE", "TRNC", "TRNA"]
pattern = "|".join(keywords)
trainee_duty = filtered_dataset[filtered_dataset["Duty ID"].str.contains(pattern, na=False)]

# Count distinct trips for each trianee
distinct_trip = trainee_duty.drop_duplicates(subset=["Duty ID", "Trip ID"])

# Group by name and coalesce all grades into a single cell
result = distinct_trip.groupby(by=["Full Name", "Duty ID", "Duty Role"])["Trip ID"].apply(lambda x: ", ".join(x)).reset_index()

# Output the result
result


,Full Name,Duty ID,Duty Role,Trip ID
0,"Curry, Wardell",411027TRNA,CUSTOMER AGENT,"27814.0, 67702.0, 77308.0"
1,"James, Byron",018901TRNE,ENGINEER,"55018.0, 25687.0, 84955.0"
2,"Jill, Tee",811027TRNE,ENGINEER,"73902.0, 33709.0, 43874.0"
3,"Ulysses, Ivan",515010TRNC,CONDUCTOR,"nan, 76806.0, 86806.0, 16806.0, 46806.0, 36806..."


### Total Off Sick Report
##### Testing to build sample of aggregated number of sick employees output

In [12]:
# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Absenteeism_Stats.xlsx", sheet_name="Sheet1")

# Filter for actual planning level
dataset = dataset[dataset["Planning Level"] == "Actual"]

# Filter for SICK absence
dataset = dataset[dataset["Duty Allocation ID"] == "SICK"]

# Drop duplicates by specific columns
dataset = dataset.drop_duplicates(subset=["Personnel ID", "Duty Allocation ID"])

# Convert "date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday"s operational data
filtered_dataset = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

filtered_dataset

,Date,Planning Level,Forename,Surname,Personnel ID,Allocation Class,Duty Allocation ID,Allocation Start Time,Allocation End Time,Payroll Code,Role
0,2025-05-21,Actual,Alyssa,Perkins,282645103,Absence,SICK,05.03.2025 00:00:00,05.03.2025 23:59:59,SICK LEAVE,Conductor
406368,2025-05-21,Actual,Ruth,Lee,377208887,Absence,SICK,05.03.2025 00:00:00,05.03.2025 23:59:59,SICK LEAVE,Engineer
406370,2025-05-21,Actual,Nicholas,Chavez,152684322,Absence,SICK,05.03.2025 00:00:00,05.03.2025 23:59:59,SICK LEAVE,Engineer
406372,2025-05-21,Actual,Nathan,Martinez,429161962,Absence,SICK,05.03.2025 00:00:00,05.03.2025 23:59:59,SICK LEAVE,Agent


### Crewing Levels Report
##### Testing to build sample of crewing level tables which is requires understanding how many unallocated duties there are (vacancies) against how many crew are allocated to the spareboard (available to fill the vacancy)
##### Part 1 - script to retrieve data on spares
##### Part 2 - script to retrieve data on vacancies/unallocated duties

In [13]:
# PART 1

# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Duty_Crew_Stats.xlsx", sheet_name="Sheet1")

# Isolate SPARE crew either through Duty Class == "Spare" 
spares = dataset[dataset["Duty Class"] == "SPARE"]

# Filter for schedule planning level, and dedupe to ensure there is only 1 row of a unique employee for each spare duty
spares_scheduled = spares[spares["Planning Level"] == "Schedule"].drop_duplicates(subset=["Personnel ID"])

# Convert "date" column to datetime
spares_scheduled["Date"] = pd.to_datetime(spares_scheduled["Date"])

# Get yesterday's date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday"s operational data
spares_scheduled = spares_scheduled[spares_scheduled["Date"] == pd.Timestamp(yesterday)]

spares_scheduled.head()

,Date,Depot,Designation ID,Duty Role,Duty ID,Duty Start Time,Duty End Time,Trip ID,Start Station,Start Station Name,End Station,End Station Name,Route,Route Name,Duty Allocation ID,Planning Level,Forename,Surname,Personnel ID,Roster,Allocated Role,Duty Class,Allocation Start Time,Allocation End Time
88179,2025-05-21,Willowbrook,11A,ENGINEER,SPARE,03:25,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,SPARE,Schedule,Lutz,Jessica,242965891.0,WB-ENGINEER-S,Engineer,SPARE,11:21:00,14:30:00
88180,2025-05-21,Willowbrook,11A,ENGINEER,SPARE,03:25,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,SPARE,Schedule,Murillo,Kathy,563947369.0,WB-ENGINEER-S,Engineer,SPARE,22:41:33,14:30:00
88181,2025-05-21,Willowbrook,11A,ENGINEER,SPARE,03:25,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,SPARE,Schedule,Taylor,Shane,386419251.0,WB-ENGINEER-S,Engineer,SPARE,00:00:21,14:30:00
88182,2025-05-21,Willowbrook,11A,ENGINEER,SPARE,03:25,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,SPARE,Schedule,Morrison,Julie,622951859.0,WB-ENGINEER-S,Engineer,SPARE,03:31:41,14:30:00
88183,2025-05-21,Willowbrook,11A,ENGINEER,SPARE,03:25,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,SPARE,Schedule,Sims,Lynn,384795340.0,WB-CONDUCTOR-S,Conductor,SPARE,07:55:19,14:30:00


In [14]:
# PART 2

# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Duty_Crew_Stats.xlsx", sheet_name="Sheet1")

# Get duties with no personnel allocated to it a.k.a. a vacancy
unallocated_duties = dataset[(dataset["Duty ID"] != "SPARE") & (dataset["Personnel ID"].isna())]

# Dedupe duty abbreviations so that there is only 1 row for each unique duty
unallocated_duties = unallocated_duties.drop_duplicates(subset="Duty ID")

# Convert 'date' column to datetime
unallocated_duties["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday's date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday's operational data
unallocated_duties = unallocated_duties[unallocated_duties["Date"] == pd.Timestamp(yesterday)]

unallocated_duties.head()

,Date,Depot,Designation ID,Duty Role,Duty ID,Duty Start Time,Duty End Time,Trip ID,Start Station,Start Station Name,End Station,End Station Name,Route,Route Name,Duty Allocation ID,Planning Level,Forename,Surname,Personnel ID,Roster,Allocated Role,Duty Class,Allocation Start Time,Allocation End Time
19,2025-05-21,Willowbrook,UP04,ENGINEER,152205EN,16:13,02:07,NaN,UN,Union Station,UN,Union Station,UP,Union Pearson Express,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2025-05-21,Willowbrook,04A,AGENT,922014CS,16:58,00:14,49139.0,UN,Union Station,WB<,Willowbrook,LW,Lakeshore West,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,2025-05-21,Willowbrook,12A,CONDUCTOR,522003CN,13:28,23:43,NaN,OS,Oshawa Station,OS,Oshawa Station,LE,Lakeshore East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,2025-05-21,Willowbrook,13A,CONDUCTOR,121004CN,06:27,16:13,49017.0,OS,Oshawa Station,UN,Union Station,LE,Lakeshore East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2025-05-21,Willowbrook,23A,CONDUCTOR,022017CN,18:28,01:07,NaN,OS,Oshawa Station,OS,Oshawa Station,LE,Lakeshore East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Spare Qualifications Report
##### Testing to build sample of spare qualifications table, outputting the route qualifications that each spare employee holds


In [15]:
# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Employee_Stats.xlsx", sheet_name="Sheet1")

# Convert "date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Concat for full name field
dataset["Full Name"] = dataset["Surname"] + ", " + dataset["Forename"]

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday"s operational data
qualifications = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

# Create df for only personnel with "SPARE" duty abbreviation and drop duplicate names
qualifications = qualifications[qualifications["Duty ID"] == "SPARE"]

# Isolate for qualifications dataframe
qualifications = qualifications[["Full Name", "Allocated Start Time", "Allocated End Time", "Employee Group", "Employee Qualification"]].dropna()

# Exclude specific qualifications
qualifications = qualifications[~qualifications["Employee Qualification"].
                                isin([item for item in qualifications["Employee Qualification"] 
                                      if any(substring in item for substring in exclude_qualifications)])]

# Split value of time fields to get times then coalesce into a range
qualifications["Allocated Start Time"] = qualifications["Allocated Start Time"].str[-8:-3]
qualifications["Allocated End Time"] = qualifications["Allocated End Time"].str[-8:-3]
qualifications["Allocated Time"] = qualifications["Allocated Start Time"] + " - " + qualifications["Allocated End Time"]

# Count distinct qualifications for each personnel
qualifications = qualifications.drop_duplicates(subset=["Full Name", "Employee Qualification"])

# Group by name and coalesce all qualifications into a single cell
result = qualifications.groupby(by=["Full Name", "Allocated Time", "Employee Group"])["Employee Qualification"].apply(lambda x: ", ".join(x)).reset_index()

# Output the result
result

,Full Name,Allocated Time,Employee Group,Employee Qualification
0,"Anderson, Joshua",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
1,"Arnold, Eric",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
2,"Black, Renee",00:00 - 23:59,Engineer,"CAN, GO, UP, UPE"
3,"Bradshaw, Emily",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, GO, GT, KIT, LON..."
4,"Brewer, April",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
5,"Campbell, Melvin",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
6,"Clarke, Eric",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
7,"Cooper, Zoe",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
8,"Ellis, Nicole",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
9,"Estrada, Charles",00:00 - 23:59,Conductor,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."


### Spare Crew Utilization Report
##### Testing to output table that shows which spare crew was utilized as well as details on how they were utilized on a trip level basis, requiring the comparison on the data level from the planned duty against the scheduled duty to highlight a duty change


In [16]:
# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Duty_Crew_Stats.xlsx", sheet_name="Sheet1")

# Concat for full name field
dataset["Full Name"] = dataset["Surname"] + ", " + dataset["Forename"]

dataset["Duty Time"] = dataset["Duty Start Time"].str.strip() + " - " + dataset["Duty End Time"].str.strip()

# Create time of utilization column
dataset["Time of Utilization"] = dataset["Allocation Start Time"]

# Convert "Date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday"s operational data
filtered_dataset = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

# Filter names who have "SPARE" in "schedule" and "STANDARD DUTY" in "actual" 
filtered_names = filtered_dataset[(filtered_dataset["Planning Level"] == "Schedule") & (filtered_dataset["Duty Class"] == "SPARE")]["Full Name"]
filtered_names = filtered_names[filtered_names.isin(filtered_dataset[(filtered_dataset["Planning Level"] == "Actual") & (filtered_dataset["Duty Class"] == "STANDARD DUTY")]["Full Name"])]

# Output dataframe that filters for those who were scheduled as Spare, but then are given a standard duty in Actual, showcasing a utilization of a spare
spares_utilized = filtered_dataset[filtered_dataset["Full Name"].isin(filtered_names)]

# Drop duplicates based on crew name and planning level
spares_utilized = spares_utilized.drop_duplicates(subset=["Personnel ID", "Planning Level", "Trip ID"])

# Filter for actual planning level
spares_utilized = spares_utilized[spares_utilized["Planning Level"] == "Actual"]

# Remove floating zeroes after filtering planning level to actual (as before filter there is NA instances, making unable to convert dtype to int to remove zeroes)
spares_utilized["Trip ID"] = spares_utilized["Trip ID"].astype(int)
spares_utilized["Trip ID"] = spares_utilized["Trip ID"].astype(str)

# Groupby relevant fields to coalesce trips into single row
result = spares_utilized.groupby(by=["Duty Class", "Full Name", "Personnel ID", "Duty Role", "Route",
                                     "Duty Time", "Time of Utilization", "Designation ID"])['Trip ID'].apply(lambda x: ', '.join(x)).reset_index()

# Set up new df to get planned duty, first drop duplicates based on crew name and duty
spares_schedule_actual = filtered_dataset.drop_duplicates(subset=["Personnel ID", "Duty ID"])

# Coalesce to get scheduled duty vs actual duty
spares_schedule_actual = spares_schedule_actual.groupby(by=["Personnel ID"])["Duty ID"].apply(lambda x: ', '.join(x)).reset_index()

# Join tables together to the get utilization data as well as schedule v actual duty abbreviation 
result = pd.merge(result, spares_schedule_actual, on="Personnel ID", how="inner")

# Set up new df to get spare allocation begin time to get which allocation time the spare was allocated to
spares_allocated= filtered_dataset[["Personnel ID", "Allocation Start Time"]][filtered_dataset["Planning Level"] == "Schedule"]

# Join tables together to the get utilization data as well as spare allocation begin time
result = pd.merge(result, spares_allocated, on="Personnel ID", how="inner")

result.T

,0,1
Duty Class,STANDARD DUTY,STANDARD DUTY
Full Name,"Clancy, Reece","Reacher, Jade"
Personnel ID,860116865.0,467916390.0
Duty Role,CONDUCTOR,CONDUCTOR
Route,RH,RH
Duty Time,03:25 - 14:30,03:25 - 14:30
Time of Utilization,03:25:00,03:25:00
Designation ID,40A,51A
Trip ID,"15006, 15091","45819, 35222"
Duty ID,"SPARE, 411006CN","SPARE, 699999CN"


### Standby Crew Utilization Report
##### Testing to output table that shows which standby crew was utilized as well as details on how they were utilized on a trip level basis, requiring the comparison on the data level from the planned duty against the scheduled duty to highlight a duty change


In [17]:
# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Duty_Crew_Stats.xlsx", sheet_name="Sheet1")

# Concat for full name field
dataset["Full Name"] = dataset["Surname"] + ", " + dataset["Forename"]

dataset["Duty Time"] = dataset["Duty Start Time"].str.strip() + " - " + dataset["Duty End Time"].str.strip()

# Create time of utilization column
dataset["Time of Utilization"] = dataset["Allocation Start Time"]

# Convert "Date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday"s operational data
filtered_dataset = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

# Filter names who have "STANDBY" in "schedule" and "STANDARD DUTY" in "actual" 
filtered_names = filtered_dataset[(filtered_dataset["Planning Level"] == "Schedule") & (filtered_dataset["Duty Class"] == "STANDBY")]["Full Name"]
filtered_names = filtered_names[filtered_names.isin(filtered_dataset[(filtered_dataset["Planning Level"] == "Actual") & (filtered_dataset["Duty Class"] == "STANDARD DUTY")]["Full Name"])]

# Output dataframe that filters for those who were scheduled as Spare, but then are given a standard duty in Actual, showcasing a utilization of a spare
standby_utilized = filtered_dataset[filtered_dataset["Full Name"].isin(filtered_names)]

# Drop duplicates based on crew name and planning level
standby_utilized = standby_utilized.drop_duplicates(subset=["Personnel ID", "Planning Level", "Trip ID"])

# Filter for actual planning level
standby_utilized = standby_utilized[standby_utilized["Planning Level"] == "Actual"]

# Remove floating zeroes after filtering planning level to actual (as before filter there is NA instances, making unable to convert dtype to int to remove zeroes)
standby_utilized["Trip ID"] = standby_utilized["Trip ID"].astype(int)
standby_utilized["Trip ID"] = standby_utilized["Trip ID"].astype(str)

# Groupby relevant fields to coalesce trips into single row
result = standby_utilized.groupby(by=["Duty Class", "Full Name", "Personnel ID", "Duty Role", "Route",
                                     "Duty Time", "Time of Utilization", "Designation ID"])['Trip ID'].apply(lambda x: ', '.join(x)).reset_index()

# Set up new df to get planned duty, first drop duplicates based on crew name and duty
standby_schedule_actual = filtered_dataset.drop_duplicates(subset=["Personnel ID", "Duty ID"])

# Coalesce to get scheduled duty vs actual duty
standby_schedule_actual = standby_schedule_actual.groupby(by=["Personnel ID"])["Duty ID"].apply(lambda x: ', '.join(x)).reset_index()

# Join tables together to the get utilization data as well as schedule v actual duty abbreviation 
result = pd.merge(result, standby_schedule_actual, on="Personnel ID", how="inner")

# Set up new df to get spare allocation begin time to get which allocation time the spare was allocated to
standby_allocated= filtered_dataset[["Personnel ID", "Allocation Start Time"]][filtered_dataset["Planning Level"] == "Schedule"]

# Join tables together to the get utilization data as well as spare allocation begin time
result = pd.merge(result, standby_allocated, on="Personnel ID", how="inner")

result.T

,0,1
Duty Class,STANDARD DUTY,STANDARD DUTY
Full Name,"Chan, Sally","Grey, Gandalf"
Personnel ID,506209781.0,106008677.0
Duty Role,CONDUCTOR,CONDUCTOR
Route,LW,LW
Duty Time,03:25 - 14:30,03:25 - 14:30
Time of Utilization,03:25:00,03:25:00
Designation ID,11A,22A
Trip ID,"23048, 84910","54012, 94872"
Duty ID,"411006CN, STANDBY-01CN, 611741CN","STANDBY-02CN, 818892CN"


### Standby Crew Report
##### Testing to get high level data on the availability and route qualifications for crew on standby
##### Part 1 - get the crew associated to the designated standby duty abbreviation 
##### Part 2 - as qualifications sits in a different table, get the qualifications for the Engineer and Conductor assigned to the specified standby duties

In [18]:
# PART 1

# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Duty_Crew_Stats.xlsx", sheet_name="Sheet1")

# Concat for full name field
dataset["Full Name"] = dataset["Surname"] + ", " + dataset["Forename"]

# Convert "date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday"s operational data
filtered_dataset = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

# Filter for only duty abbreviations associated to the designated standby duties
filtered_dataset["Duty ID"] = filtered_dataset["Duty ID"].astype(str)

keywords = ["STANDBY-01", "STANDBY-02", "STANDBY-03", "STANDBY-04"]
pattern = "|".join(keywords)

filtered_dataset = filtered_dataset[filtered_dataset["Duty ID"].str.contains(pattern, case=False, na=False)]

# Split value of time fields to get times then coalesce into a range
filtered_dataset["Duty Start Time"] = filtered_dataset["Duty Start Time"].str[-8:]
filtered_dataset["Duty End Time"] = filtered_dataset["Duty End Time"].str[-8:]
filtered_dataset["Duty Time"] = filtered_dataset["Duty Start Time"] + " - " + filtered_dataset["Duty End Time"]

# Function to assign engineer availability status based on conditions
E_duties = ["STANDBY-01EN", "STANDBY-02EN", "STANDBY-03EN", "STANDBY-04EN"]

def E_availability(filtered_dataset, standby_duties):
    filtered_dataset["ENGINEER_Status"] = filtered_dataset.apply(lambda row: 'Available' if row['Duty ID'] in standby_duties and pd.notna(row['Full Name']) else 'Unavailable', axis=1)

    return filtered_dataset

# Function to assign conductor availability status based on conditions
C_duties = ["STANDBY-01CN", "STANDBY-02CN", "STANDBY-03CN", "STANDBY-04CN"]

def C_availablity(filtered_dataset, standby_duties):
    filtered_dataset["CONDUCTOR_Status"] = filtered_dataset.apply(lambda row: 'Available' if row['Duty ID'] in standby_duties and pd.notna(row['Full Name']) else 'Unavailable', axis=1)

    return filtered_dataset

# Function to assign customer agent availability status based on conditions
A_duties = ["STANDBY-01CS", "STANDBY-02CS", "STANDBY-03CS", "STANDBY-04CS"]

def A_availability(filtered_dataset, standby_duties):
    filtered_dataset["CUSTOMER_AGENT_Status"] = filtered_dataset.apply(lambda row: 'Available' if row['Duty ID'] in standby_duties and pd.notna(row['Full Name']) else 'Unavailable', axis=1)

    return filtered_dataset

# Store result df
result = E_availability(filtered_dataset, E_duties)
result = C_availablity(filtered_dataset, C_duties)
result = A_availability(filtered_dataset, A_duties)

result.head()

,Date,Depot,Designation ID,Duty Role,Duty ID,Duty Start Time,Duty End Time,Trip ID,Start Station,Start Station Name,End Station,End Station Name,Route,Route Name,Duty Allocation ID,Planning Level,Forename,Surname,Personnel ID,Roster,Allocated Role,Duty Class,Allocation Start Time,Allocation End Time,Full Name,Duty Time,ENGINEER_Status,CONDUCTOR_Status,CUSTOMER_AGENT_Status
88203,2025-05-21,NaN,NaN,NaN,STANDBY-01CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STANDBY-01CN,Schedule,Sally,Chan,506209781.0,WB-CONDUCTOR-STANDBY,Conductor,STANDBY,00:00:00,23:59:59,"Chan, Sally",NaN,Unavailable,Available,Unavailable
88206,2025-05-21,NaN,NaN,NaN,STANDBY-02CN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STANDBY-02CN,Schedule,Gandalf,Grey,106008677.0,WB-CONDUCTOR-STANDBY,Conductor,STANDBY,00:00:00,23:59:59,"Grey, Gandalf",NaN,Unavailable,Available,Unavailable
88209,2025-05-21,Willowbrook,11A,ENGINEER,STANDBY-01EN,2:30,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,STANDBY-01EN,Schedule,Lutz,Jessica,242965891.0,WB-ENGINEER-D,Engineer,STANDARD DUTY,0.142685,14:30:00,"Jessica, Lutz",2:30 - 14:30,Available,Unavailable,Unavailable
88210,2025-05-21,Willowbrook,11A,CONDUCTOR,STANDBY-01CN,2:30,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,STANDBY-01CN,Schedule,Murillo,Kathy,563947369.0,WB-CONDUCTOR-D,Conductor,STANDARD DUTY,0.469815,14:30:00,"Kathy, Murillo",2:30 - 14:30,Unavailable,Available,Unavailable
88211,2025-05-21,Willowbrook,11A,CUSTOMER AGENT,STANDBY-01CS,2:30,14:30,NaN,AL,Aldershot Station,AL,Aldershot Station,LW,Lakeshore West,STANDBY-01CS,Schedule,Taylor,Shane,386419251.0,WB-AGENT-D,Customer Agent,STANDARD DUTY,0.360926,14:30:00,"Shane, Taylor",2:30 - 14:30,Unavailable,Unavailable,Available


In [19]:
# PART 2

# Import modules
import pandas as pd
from datetime import datetime, timedelta

# Import dataset
dataset = pd.read_excel("../src/Rail_Employee_Stats.xlsx", sheet_name="Sheet1")

# Concat for full name field
dataset["Full Name"] = dataset["Surname"] + ", " + dataset["Forename"]

# Filter for only duty abbreviations associated to the designated standby duties
dataset["Duty ID"] = dataset["Duty ID"].astype(str)

# Convert "date" column to datetime
dataset["Date"] = pd.to_datetime(dataset["Date"])

# Get yesterday"s date
yesterday = datetime.now() - timedelta(days=1)
yesterday = yesterday.date()

# Apply filter to get yesterday"s operational data
filtered_dataset = dataset[dataset["Date"] == pd.Timestamp(yesterday)]

keywords = ["STANDBY-01", "STANDBY-02", "STANDBY-03", "STANDBY-04"]
pattern = "|".join(keywords)

filtered_dataset = filtered_dataset[filtered_dataset["Duty ID"].str.contains(pattern, case=False, na=False)]

# Build qualifications df
qualifications = filtered_dataset[["Full Name", "Duty Roster", "Duty ID", "Duty Class", "Employee Qualification"]].dropna()

# Count distinct qualifications for each personnel
qualifications = qualifications.drop_duplicates(subset=["Full Name", "Employee Qualification"])

# Group by name and coalesce all qualifications into a single cell
result = qualifications.groupby(by=["Duty Class", "Duty ID", "Full Name", "Duty Roster"])["Employee Qualification"].apply(lambda x: ", ".join(x)).reset_index()

# Output the result
result

,Duty Class,Duty ID,Full Name,Duty Roster,Employee Qualification
0,STANDBY,STANDBY-01CN,"Lopez, Nicole",WB-CONDUCTOR-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
1,STANDBY,STANDBY-01CN,"Williams, Marcus",WB-CONDUCTOR-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
2,STANDBY,STANDBY-01CS,"Brown, Charles",WB-AGENT-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
3,STANDBY,STANDBY-01EN,"Frey, Jonathan",WB-ENGINEER-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
4,STANDBY,STANDBY-02CN,"Espinoza, David",WB-CONDUCTOR-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
5,STANDBY,STANDBY-02CN,"Hamilton, Kevin",WB-CONDUCTOR-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
6,STANDBY,STANDBY-02CS,"Lewis, Misty",WB-AGENT-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
7,STANDBY,STANDBY-02EN,"Waller, Amanda",WB-ENGINEER-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
8,STANDBY,STANDBY-03CN,"Jensen, Sarah",WB-CONDUCTOR-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
9,STANDBY,STANDBY-03CN,"Reyes, Trevor",WB-CONDUCTOR-STANDBY,"AUR, BAR, BRAD, CAN, CN Cert, CN Quali, GO, GT..."
